In [1]:
import os
import numpy as np
import subprocess
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from efficientnet.keras import EfficientNetB0

In [2]:
trainDir = "/Users/ppr/Desktop/Project/Intern-Year2/Datasets/GrayDatasetsResizeAugmented/train"
valDir = "/Users/ppr/Desktop/Project/Intern-Year2/Datasets/GrayDatasetsResizeAugmented/val"
testDir = "/Users/ppr/Desktop/Project/Intern-Year2/Datasets/GrayDatasetsResizeAugmented/test"

In [3]:
imgWidth, imgHeight = 640, 640
learningRate = 0.001
numClasses = 11
batchSize = 32
epochs = 10

In [4]:
trainDatagen = ImageDataGenerator(rescale=1./255)
valDatagen = ImageDataGenerator(rescale=1./255)
testDatagen = ImageDataGenerator(rescale=1./255)

In [5]:
trainGenerator = trainDatagen.flow_from_directory(
        trainDir,
        target_size=(imgWidth, imgHeight),
        batch_size=batchSize,
        class_mode='categorical')

validationGenerator = valDatagen.flow_from_directory(
        valDir,
        target_size=(imgWidth, imgHeight),
        batch_size=batchSize,
        class_mode='categorical')

testGenerator = testDatagen.flow_from_directory(
        testDir,
        target_size=(imgWidth, imgHeight),
        batch_size=1,
        shuffle=False,
        class_mode='categorical')

Found 781 images belonging to 11 classes.
Found 147 images belonging to 11 classes.
Found 133 images belonging to 11 classes.


In [6]:
numTrainSamples = trainGenerator.samples
numValSamples = validationGenerator.samples

In [7]:
# Convert labels to one-hot encoding
trainLabels = to_categorical(trainGenerator.classes, num_classes=numClasses)
valLabels = to_categorical(validationGenerator.classes, num_classes=numClasses)

In [8]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(imgWidth, imgHeight, 3))

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(numClasses, activation='softmax'))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnet-b0 (Functiona  (None, 20, 20, 1280)      4049564   
 l)                                                              
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 11)                14091     
                                                                 
Total params: 4063655 (15.50 MB)
Trainable params: 4021639 (15.34 MB)
Non-trainable params: 42016 (164.12 KB)
_________________________________________________________________


In [10]:
for layer in base_model.layers:
    layer.trainable = False

In [11]:
optimizer = Adam(learning_rate=learningRate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True, save_weights_only=False, mode='auto')

In [13]:
model.fit(
        trainGenerator,
        steps_per_epoch=numTrainSamples // batchSize,
        epochs=epochs,
        validation_data=validationGenerator,
        validation_steps=numValSamples // batchSize,
        callbacks=[checkpoint])

Epoch 1/10


2023-05-31 12:39:50.130619: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


24/24 [==============================] - ETA: 0s - loss: 2.0377 - accuracy: 0.2951

2023-05-31 12:40:42.786206: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


24/24 [==============================] - 62s 2s/step - loss: 2.0377 - accuracy: 0.2951 - val_loss: 1.7229 - val_accuracy: 0.5156
Epoch 2/10


/Users/ppr/miniforge3/envs/sophomore/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


24/24 [==============================] - 58s 2s/step - loss: 1.4415 - accuracy: 0.5821 - val_loss: 1.3896 - val_accuracy: 0.6797
Epoch 3/10
24/24 [==============================] - 58s 2s/step - loss: 1.1414 - accuracy: 0.6956 - val_loss: 1.1831 - val_accuracy: 0.7266
Epoch 4/10
24/24 [==============================] - 59s 2s/step - loss: 0.9601 - accuracy: 0.7623 - val_loss: 1.0958 - val_accuracy: 0.7188
Epoch 5/10
24/24 [==============================] - 58s 2s/step - loss: 0.8322 - accuracy: 0.7797 - val_loss: 1.0506 - val_accuracy: 0.7031
Epoch 6/10
24/24 [==============================] - 57s 2s/step - loss: 0.7490 - accuracy: 0.8304 - val_loss: 0.9720 - val_accuracy: 0.7188
Epoch 7/10
24/24 [==============================] - 57s 2s/step - loss: 0.6676 - accuracy: 0.8478 - val_loss: 0.9227 - val_accuracy: 0.7422
Epoch 8/10
24/24 [==============================] - 57s 2s/step - loss: 0.6040 - accuracy: 0.8585 - val_loss: 0.8913 - val_accuracy: 0.7578
Epoch 9/10
24/24 [=============

In [14]:
test_loss, test_acc = model.evaluate(testGenerator, steps = testGenerator.samples // batchSize)
print('Test accuracy:', test_acc)

1/4 [======>.......................] - ETA: 0s - loss: 1.1948 - accuracy: 1.0000

2023-05-31 12:49:30.973946: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


4/4 [==============================] - 0s 91ms/step - loss: 0.5477 - accuracy: 1.0000
Test accuracy: 1.0


In [15]:
model.save('efficientnetModel.h5')

In [16]:
model.save('efficientnetModel.keras')